In [1]:
DRIVE_PATH = ''
mode = 'roberta' 
task_name = 'trip'
debug = False
config_batch_size = 1
config_lr = 1e-5 
config_epochs = 15
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import json
import sys
import torch
import random
import numpy as np
import spacy
import pandas as pd
from tqdm import tqdm
from Methods_for_zero_inference import *
model_name = 'roberta-large'
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')
from transformers import RobertaConfig,RobertaModel,AdamW
config_class = RobertaConfig
emb_class = RobertaModel
from www.utils import print_dict
import json
from collections import Counter
nlp = spacy.load('en_core_web_sm')


from spacy.lang.en import English
nlp_sen = English()
config = {"punct_chars": None}
nlp_sen.add_pipe("sentencizer", config=config)

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load presetting model

In [1]:
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
from Model_Joint import RobertaProceduralTSLMJointDummySoft
from www.dataset.ann import att_to_idx, att_to_num_classes, att_types

num_state_labels = {}
for att in att_to_idx:
    if att_types[att] == 'default':
        num_state_labels[att_to_idx[att]] = 3
    else:
        num_state_labels[att_to_idx[att]] = att_to_num_classes[att] # Location attributes fall into this since they don't have well-define pre- and post-condition yet

seed_val = 22 # Save random seed for reproducibility
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

config = config_class.from_pretrained(model_name)  
device="cuda:0"
num_attributes=len(num_state_labels)

story_loss=False
story_back=False
tslm= RobertaProceduralTSLMJointDummySoft.from_pretrained('tli8hf/unqover-roberta-large-squad', return_dict=True,num_attributes=num_attributes,labels_per_att=num_state_labels,story_loss=story_loss,story_back=story_back).to(device)
tslm_optimizer = AdamW(tslm.parameters(), lr=config_lr)

### aNLI

In [4]:
dev_dataset = np.load('../Labeler/Participant_Extraction/Final_Participant/ANLI_Participant_IJCAI.npy',allow_pickle = True)

In [5]:
dev_dataset = orderdata(dev_dataset)

In [6]:
Maxlength=100
dev_dataset=add_input_feature_joint_soft_dummy(dev_dataset,tokenizer,Maxlength)

Add input feature joint


100%|██████████| 1532/1532 [00:06<00:00, 244.04it/s]


In [7]:
for sample in dev_dataset:
    if len(sample['goal_sol_1'])!=len(sample['goal_sol_2']):
        Maxlen=max(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Minlen=min(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Gap=Maxlen-Minlen
        if len(sample['goal_sol_1'])<len(sample['goal_sol_2']):
            for input_rep in sample['anli_input_1']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_1']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
        else:
            for input_rep in sample['anli_input_2']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_2']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))

#### zero inference on aNLI
**file_path** contains the folder directory of model

**model_name** contains the model name

**sentence_tag** control the model is **sentencen-centric** or not

In [8]:
sentence_tag = True
task="test"
file_path="The folder directory of model"

model_name = "Model name"
 
Dir_file = model_name
FILE_PATH = os.path.join(file_path,Dir_file)
best_accuracy_model = find_best_model(FILE_PATH,task,15)
file_number=str(best_accuracy_model)
Epoch_number=file_number
output_model_path=os.path.join(FILE_PATH, file_number)

### zero inference 

tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))

tslm.zero_grad()
tslm.eval()
for layer in tslm.precondition_classifiers:
    layer.eval()
for layer in tslm.effect_classifiers:
    layer.eval()   

sol_label_summary=None
sol_pred_summary=None
all_summary=[]
with torch.no_grad():
    for index,batch in enumerate(tqdm(dev_dataset)):
        try:
            sol_label = np.array([[batch['label']-1]])

            sol_pred,batch_summary = predict_from_zero_shot_dummy(tslm,batch,device,sentence_tag)
            batch_summary['index']=index
            sol_pred=np.array([[sol_pred]])

#             Summary the result
            if sol_label_summary is None:
                sol_label_summary = sol_label
            else:
                sol_label_summary = np.concatenate((sol_label_summary, sol_label), axis=0)

            if sol_pred_summary is None:
                sol_pred_summary = sol_pred
            else:
                sol_pred_summary = np.concatenate((sol_pred_summary, sol_pred), axis=0)


            all_summary.append(batch_summary)
        except:
            torch.cuda.empty_cache()
            print("Can not find entity at {}".format(index))

# Get the result
metrics = [(accuracy_score, 'accuracy'),(f1_score, 'f1')]
metr_stories = compute_metrics(sol_pred_summary.flatten(), sol_label_summary.flatten(), metrics)
print(metr_stories['accuracy'])



### PIQA

In [10]:
dev_dataset = np.load('../Labeler/Participant_Extraction/Final_Participant/PIQA_Participant_IJCAI.npy',allow_pickle = True)

In [11]:
dev_dataset = orderdata(dev_dataset)

In [12]:
Maxlength=270
dev_dataset=add_input_feature_joint_soft_dummy(dev_dataset,tokenizer,Maxlength)
for sample in dev_dataset:
    if len(sample['goal_sol_1'])!=len(sample['goal_sol_2']):
        Maxlen=max(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Minlen=min(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Gap=Maxlen-Minlen
        if len(sample['goal_sol_1'])<len(sample['goal_sol_2']):
            for input_rep in sample['anli_input_1']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_1']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
        else:
            for input_rep in sample['anli_input_2']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_2']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))

Add input feature joint


100%|██████████| 1838/1838 [00:13<00:00, 132.04it/s]


#### zero inference on PIQA
**file_path** contains the folder directory of model

**model_name** contains the model name

**sentence_tag** control the model is **sentencen-centric** or not

In [ ]:
sentence_tag = True


task="test"
file_path="The folder directory of model"

model_name = "Model name"
 
Dir_file = model_name
FILE_PATH = os.path.join(file_path,Dir_file)
best_accuracy_model = find_best_model(FILE_PATH,task,15)
file_number=str(best_accuracy_model)
Epoch_number=file_number
output_model_path=os.path.join(FILE_PATH, file_number)

### zero inference 

tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))

tslm.zero_grad()
tslm.eval()
for layer in tslm.precondition_classifiers:
    layer.eval()
for layer in tslm.effect_classifiers:
    layer.eval()   

sol_label_summary=None
sol_pred_summary=None
all_summary=[]
with torch.no_grad():
    for index,batch in enumerate(tqdm(dev_dataset)):
        try:
            sol_label = np.array([[batch['label']]])

            sol_pred,batch_summary = predict_from_zero_shot_dummy(tslm,batch,device,sentence_tag)
            batch_summary['index']=index
            sol_pred=np.array([[sol_pred]])

#             Summary the result
            if sol_label_summary is None:
                sol_label_summary = sol_label
            else:
                sol_label_summary = np.concatenate((sol_label_summary, sol_label), axis=0)

            if sol_pred_summary is None:
                sol_pred_summary = sol_pred
            else:
                sol_pred_summary = np.concatenate((sol_pred_summary, sol_pred), axis=0)


            all_summary.append(batch_summary)
        except:
            torch.cuda.empty_cache()
            print("Can not find entity at {}".format(index))

# Get the result
metrics = [(accuracy_score, 'accuracy'),(f1_score, 'f1')]
metr_stories = compute_metrics(sol_pred_summary.flatten(), sol_label_summary.flatten(), metrics)
print(metr_stories['accuracy'])



## ROCStoires

In [14]:
dev_dataset = np.load('../Labeler/Participant_Extraction/Final_Participant/ROC_Participant_IJCAI.npy',allow_pickle = True)

In [15]:
dev_dataset = orderdata(dev_dataset)

In [16]:
Maxlength=100
dev_dataset=add_input_feature_joint_soft_dummy(dev_data,tokenizer,Maxlength)
for sample in dev_dataset:
    if len(sample['goal_sol_1'])!=len(sample['goal_sol_2']):
        Maxlen=max(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Minlen=min(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Gap=Maxlen-Minlen
        if len(sample['goal_sol_1'])<len(sample['goal_sol_2']):
            for input_rep in sample['anli_input_1']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_1']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
        else:
            for input_rep in sample['anli_input_2']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_2']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))

Add input feature joint


100%|██████████| 1871/1871 [00:18<00:00, 100.97it/s]


#### zero inference on ROCStories
**file_path** contains the folder directory of model

**model_name** contains the model name

**sentence_tag** control the model is **sentencen-centric** or not

In [ ]:
sentence_tag = True
task="test"
file_path="The folder directory of model"

model_name = "Model name"
 
Dir_file = model_name
FILE_PATH = os.path.join(file_path,Dir_file)
best_accuracy_model = find_best_model(FILE_PATH,task,15)
file_number=str(best_accuracy_model)
Epoch_number=file_number
output_model_path=os.path.join(FILE_PATH, file_number)

### zero inference 

tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))

tslm.zero_grad()
tslm.eval()
for layer in tslm.precondition_classifiers:
    layer.eval()
for layer in tslm.effect_classifiers:
    layer.eval()   

sol_label_summary=None
sol_pred_summary=None
all_summary=[]
with torch.no_grad():
    for index,batch in enumerate(tqdm(dev_dataset)):
        try:
            sol_label = np.array([[batch['label']-1]])

            sol_pred,batch_summary = predict_from_zero_shot_dummy(tslm,batch,device,sentence_tag)
            batch_summary['index']=index
            sol_pred=np.array([[sol_pred]])

#             Summary the result
            if sol_label_summary is None:
                sol_label_summary = sol_label
            else:
                sol_label_summary = np.concatenate((sol_label_summary, sol_label), axis=0)

            if sol_pred_summary is None:
                sol_pred_summary = sol_pred
            else:
                sol_pred_summary = np.concatenate((sol_pred_summary, sol_pred), axis=0)


            all_summary.append(batch_summary)
        except:
            torch.cuda.empty_cache()
            print("Can not find entity at {}".format(index))

# Get the result
metrics = [(accuracy_score, 'accuracy'),(f1_score, 'f1')]
metr_stories = compute_metrics(sol_pred_summary.flatten(), sol_label_summary.flatten(), metrics)
print(metr_stories['accuracy'])

## CODAH

In [17]:
dev_dataset = np.load('../Labeler/Participant_Extraction/Final_Participant/CODAH_Participant_IJCAI.npy',allow_pickle = True)

In [18]:
for sample in dev_dataset:
    common_list=set(sample['entity'][0])&set(sample['entity'][1])&set(sample['entity'][2])&set(sample['entity'][3])
    common_list=sorted(list(common_list))
    sample['common_entity']=len(common_list)
    for index,entity in enumerate(sample['entity']):
        sample['entity'][index]=common_list

In [24]:
Maxlength=105
dev_dataset=add_input_feature_joint_soft_dummy_codah(dev_dataset,tokenizer,Maxlength)

Add input feature joint


100%|██████████| 2776/2776 [00:14<00:00, 192.19it/s]


#### zero inference on CODAH
**file_path** contains the folder directory of model

**model_name** contains the model name

**sentence_tag** control the model is **sentencen-centric** or not

In [ ]:
sentence_tag = True
task="test"
file_path="The folder directory of model"

model_name = "Model name"
 
Dir_file = model_name
FILE_PATH = os.path.join(file_path,Dir_file)
best_accuracy_model = find_best_model(FILE_PATH,task,15)
file_number=str(best_accuracy_model)
Epoch_number=file_number
output_model_path=os.path.join(FILE_PATH, file_number)

### zero inference 

tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))

tslm.zero_grad()
tslm.eval()
for layer in tslm.precondition_classifiers:
    layer.eval()
for layer in tslm.effect_classifiers:
    layer.eval()   

sol_label_summary=None
sol_pred_summary=None
all_summary=[]
with torch.no_grad():
    for index,batch in enumerate(tqdm(dev_dataset)):
        try:
            sol_label = np.array([[batch['label']-1]])

            sol_pred,batch_summary = predict_from_zero_shot_dummy_codah(tslm,batch,device,sentence_tag)
            batch_summary['index']=index
            sol_pred=np.array([[sol_pred]])

#             Summary the result
            if sol_label_summary is None:
                sol_label_summary = sol_label
            else:
                sol_label_summary = np.concatenate((sol_label_summary, sol_label), axis=0)

            if sol_pred_summary is None:
                sol_pred_summary = sol_pred
            else:
                sol_pred_summary = np.concatenate((sol_pred_summary, sol_pred), axis=0)


            all_summary.append(batch_summary)
        except:
            torch.cuda.empty_cache()
            print("Can not find entity at {}".format(index))

# Get the result
metrics = [(accuracy_score, 'accuracy'),(f1_score, 'f1')]
metr_stories = compute_metrics(sol_pred_summary.flatten(), sol_label_summary.flatten(), metrics)
print(metr_stories['accuracy'])

## RICA

#### zero inference on RICA
Notice: All participants in RICA are noval desigend new term, so we don't include participants annotatoin on RICA.
**file_path** contains the folder directory of model

**model_name** contains the model name

**sentence_tag** control the model is **sentencen-centric** or not

In [ ]:
test_sentences=[]
with open('../Target_task/test_sentences.txt', 'r') as f:
    for line in f:
        test_sentences.append(line.replace('\n',''))

test_sentences_masks = []
with open('../Target_task/test_sentences_m.txt', 'r') as f:
    for line in f:
        test_sentences_masks.append(line.replace('\n',''))

import ast
test_sentences_mask_pairs = []
with open('../Target_task/config.txt', 'r') as f:
    for line in f:
        test_sentences_mask_pairs.append(ast.literal_eval(line))

In [ ]:
dev_data = []
count = 0
for correct_sen,correct_word,mask_pair in zip(test_sentences,test_sentences_masks,test_sentences_mask_pairs):
    mask_pair.remove(correct_word)
    incorrect_word = mask_pair[0]
    wrong_sen = correct_sen.replace(correct_word,incorrect_word)
    sample_story = {}
    sample_story['goal_sol_1'] = [_.strip() for _ in correct_sen.split(",")]
    sample_story['word_list'] = [correct_word,incorrect_word]
    sample_story['goal_sol_2'] = [_.strip() for _ in wrong_sen.split(",")]
    sample_story['entity_1']=[]
    sample_story['entity_2']=[]
    sample_story['common_entity']=0
    sample_story['label']=0    
    dev_data.append(sample_story)

In [ ]:
Maxlength=100
dev_dataset=add_input_feature_joint_soft_dummy(dev_data,tokenizer,Maxlength)

In [ ]:
for sample in dev_dataset:
#     if (len(sample['entity_1'])+len(sample['entity_2']))==0:
#         count+=1
    if len(sample['goal_sol_1'])!=len(sample['goal_sol_2']):
        print(1)
        Maxlen=max(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Minlen=min(len(sample['goal_sol_1']),len(sample['goal_sol_2']))
        Gap=Maxlen-Minlen
        if len(sample['goal_sol_1'])<len(sample['goal_sol_2']):
            for input_rep in sample['anli_input_1']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_1']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
        else:
            for input_rep in sample['anli_input_2']:
                for key in ['input_ids','attention_mask','timestamp_id']:
                    input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))
            input_rep=sample['anli_dummy_2']
            for key in ['input_ids','attention_mask','timestamp_id']:
                input_rep[key]=torch.cat((input_rep[key],torch.zeros([Gap,Maxlength])))

In [ ]:
sentence_tag = True
task="test"
file_path="The folder directory of model"

model_name = "Model name"
 
Dir_file = model_name
FILE_PATH = os.path.join(file_path,Dir_file)
best_accuracy_model = find_best_model(FILE_PATH,task,15)
file_number=str(best_accuracy_model)
Epoch_number=file_number
output_model_path=os.path.join(FILE_PATH, file_number)

### zero inference 

tslm = torch.load(os.path.join(output_model_path, 'tslm.pth'))

tslm.zero_grad()
tslm.eval()
for layer in tslm.precondition_classifiers:
    layer.eval()
for layer in tslm.effect_classifiers:
    layer.eval()   

sol_label_summary=None
sol_pred_summary=None
all_summary=[]
with torch.no_grad():
    for index,batch in enumerate(tqdm(dev_dataset)):
        try:
            sol_label = np.array([[batch['label']-1]])

            sol_pred,batch_summary = predict_from_zero_shot_dummy(tslm,batch,device,sentence_tag)
            batch_summary['index']=index
            sol_pred=np.array([[sol_pred]])

#             Summary the result
            if sol_label_summary is None:
                sol_label_summary = sol_label
            else:
                sol_label_summary = np.concatenate((sol_label_summary, sol_label), axis=0)

            if sol_pred_summary is None:
                sol_pred_summary = sol_pred
            else:
                sol_pred_summary = np.concatenate((sol_pred_summary, sol_pred), axis=0)


            all_summary.append(batch_summary)
        except:
            torch.cuda.empty_cache()
            print("Can not find entity at {}".format(index))

# Get the result
metrics = [(accuracy_score, 'accuracy'),(f1_score, 'f1')]
metr_stories = compute_metrics(sol_pred_summary.flatten(), sol_label_summary.flatten(), metrics)
print(metr_stories['accuracy'])
